In [1]:
# Step1 load MNITST data
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data", one_hot=True)

/home/mzx/.local/lib/python3.5/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [2]:
import numpy as np
import tensorflow as tf

### Build the DNN network
| Layer | Structure |  BN | Activation |
|:-----:|:---------:|:---:|:----------:|
| Input |[None, 784]| --- |    ---     |
| FC1   |[None, 512]| 512 |    ReLU    |
| FC2   |[None, 256]| 256 |    ReLU    |
| FC3   |[None, 128]| 128 |    ReLU    |
| Output|[None, 10] | --- |    tanh    |  

In [3]:
# Placeholders
x = tf.placeholder(tf.float32, shape=[None, 784])
y_ = tf.placeholder(tf.float32, shape=[None, 10]) # one-hot 10-dimensional vector

In [4]:
# Layers
# Refer to tf.nn.moments
# FC1
l_FC1 = 512
W_FC1 = tf.Variable(tf.truncated_normal(shape=[784,l_FC1], stddev=0.1))
b_FC1 = tf.Variable(tf.constant(0.1,shape=[l_FC1]))

# batch_norm_l1
x_FC1 = tf.matmul(x, W_FC1)+b_FC1
axis = list(range(len(x_FC1.get_shape()) - 1))
mean_FC1,var_FC1 = tf.nn.moments(x_FC1, axis)
gamma_FC1 = tf.Variable(tf.constant(0.1, shape=mean_FC1.get_shape()))
beta_FC1 = tf.Variable(tf.constant(0.1, shape=mean_FC1.get_shape()))
y_bn1 = tf.nn.batch_normalization(
    x = x_FC1,
    mean = mean_FC1,
    variance = var_FC1,
    offset = beta_FC1,
    scale = gamma_FC1,
    variance_epsilon = 1e-5,
    name= 'BN_FC1')
y_FC1 = tf.nn.relu(y_bn1)

In [5]:
# FC2
l_FC2 = 256
W_FC2 = tf.Variable(tf.truncated_normal(shape=[l_FC1,l_FC2], stddev=0.1))
b_FC2 = tf.Variable(tf.constant(0.1,shape=[l_FC2]))

# batch_norm_l2
x_FC2 = tf.matmul(y_FC1, W_FC2)+b_FC2
axis = list(range(len(x_FC2.get_shape()) - 1))
mean_FC2,var_FC2 = tf.nn.moments(x_FC2, axis)
gamma_FC2 = tf.Variable(tf.constant(0.1, shape=mean_FC2.get_shape()))
beta_FC2 = tf.Variable(tf.constant(0.1, shape=mean_FC2.get_shape()))
y_bn2 = tf.nn.batch_normalization(
    x = x_FC2,
    mean = mean_FC2,
    variance = var_FC2,
    offset = beta_FC2,
    scale = gamma_FC2,
    variance_epsilon = 1e-5,
    name= 'BN_FC2')
y_FC2 = tf.nn.relu(y_bn2)

In [6]:
# FC3
l_FC3 = 128
W_FC3 = tf.Variable(tf.truncated_normal(shape=[l_FC2,l_FC3], stddev=0.1))
b_FC3 = tf.Variable(tf.constant(0.1,shape=[l_FC3]))

# batch_norm_l3
x_FC3 = tf.matmul(y_FC2, W_FC3)+b_FC3
axis = list(range(len(x_FC3.get_shape()) - 1))
mean_FC3,var_FC3 = tf.nn.moments(x_FC3, axis)
gamma_FC3 = tf.Variable(tf.constant(0.1, shape=mean_FC3.get_shape()))
beta_FC3 = tf.Variable(tf.constant(0.1, shape=mean_FC3.get_shape()))
y_bn3 = tf.nn.batch_normalization(
    x = x_FC3,
    mean = mean_FC3,
    variance = var_FC3,
    offset = beta_FC3,
    scale = gamma_FC3,
    variance_epsilon = 1e-5,
    name= 'BN_FC3')
y_FC3 = tf.nn.relu(y_bn3)

In [7]:
# Softmax layer
l_soft = 10
W_s = tf.Variable(tf.truncated_normal(shape=[l_FC3,l_soft], stddev=0.1))
b_s = tf.Variable(tf.constant(0.1,shape=[l_soft]))

x_s = tf.matmul(y_FC3, W_s)+b_s
y = tf.nn.tanh(x_s)

#### Loss function: cross_entropy

In [8]:
# loss function
# Note: labels and logits should be clearly assigned
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y))

In [9]:
# Applying the steepest gradient descent, with a step of 0.5, to descend the cross entropy
# Tensorflow then add new operations to the computation graph, including ones to compute gradients, 
# compute parameters update steps, and apply update steps to the parameters.
train_step = tf.train.GradientDescentOptimizer(0.5).minimize(cross_entropy)

### Training the network

In [10]:
# Initialize a session
sess = tf.InteractiveSession()
sess.run(tf.global_variables_initializer())

In [11]:
# Training the model by repeatedly running train_step
import time 
epochs = 100
batchsize= 100
for i in range(epochs):
    batch = mnist.train.next_batch(batchsize)
    train_step.run(feed_dict={x: batch[0], y_: batch[1]})
    if (i+1) % 10 == 0:
        timestamp = time.strftime('%Y-%m-%d: %H:%M:%S', time.localtime(time.time()))
        print("[%s]: Epochs %d" % (timestamp, i+1))

[2018-01-24: 10:39:49]: Epochs 10
[2018-01-24: 10:39:49]: Epochs 20
[2018-01-24: 10:39:49]: Epochs 30
[2018-01-24: 10:39:49]: Epochs 40
[2018-01-24: 10:39:49]: Epochs 50
[2018-01-24: 10:39:49]: Epochs 60
[2018-01-24: 10:39:49]: Epochs 70
[2018-01-24: 10:39:49]: Epochs 80
[2018-01-24: 10:39:49]: Epochs 90
[2018-01-24: 10:39:49]: Epochs 100
